In [ ]:
# check if there are duplictaes in the clinical notes

import numpy as np
from collections import Counter

In [ ]:
import pickle

df_train = pickle.load(open('/home/thetaphipsi/MasterAI/src/CNEP/src/data/mimic3/full_train_data.pickle', 'rb'))
df_val = pickle.load(open('/home/thetaphipsi/MasterAI/src/CNEP/src/data/mimic3/full_val_data.pickle', 'rb'))
df_test = pickle.load(open('/home/thetaphipsi/MasterAI/src/CNEP/src/data/mimic3/full_test_data.pickle', 'rb'))

In [ ]:
sequence_length = None

notes_train = [n[:sequence_length] for n in df_train['notes']]
notes_val = [n[:sequence_length] for n in df_val['notes']]
notes_test = [n[:sequence_length] for n in df_test['notes']]

In [ ]:
def checkIfDuplicates(listOfElems):
    ''' Check if given list contains any duplicates '''
    if len(listOfElems) == len(set(listOfElems)):
        return False
    else:
        return True

checkIfDuplicates(notes_train), checkIfDuplicates(notes_val), checkIfDuplicates(notes_test)

In [ ]:
len(notes_train), len(set(notes_train)), len(notes_val), len(set(notes_val)), len(notes_test), len(set(notes_test))

In [ ]:
def getDupes(l):
    seen = set()
    seen_add = seen.add
    for x in l:
        if x in seen or seen_add(x):
            yield x

In [ ]:
dupl_train = list(set([n for n in getDupes(notes_train)]))
dupl_val = list(set([n for n in getDupes(notes_val)]))
dupl_test = list(set([n for n in getDupes(notes_test)]))

In [ ]:
assert dupl_train[0] in notes_train
assert dupl_val[0] in notes_val
assert dupl_test[0] in notes_test

In [ ]:
checkIfDuplicates(set(dupl_train))

In [ ]:
indices_train = [[i for i, x in enumerate(notes_train) if x == d] for d in set(dupl_train)]
indices_val = [[i for i, x in enumerate(notes_val) if x == d] for d in set(dupl_val)]
indices_test = [[i for i, x in enumerate(notes_test) if x == d] for d in set(dupl_test)]

In [ ]:
indices_train[:5]

In [ ]:
len(indices_train), len(indices_val), len(indices_test)

In [ ]:
#[df_train['labels'][i] for i in indices_train]

In [ ]:
allduplicates_train = [len([i for i, x in enumerate(notes_train) if x == dupl_train[j]]) for j in range(len(dupl_train))]
allduplicates_val = [len([i for i, x in enumerate(notes_val) if x == dupl_val[j]]) for j in range(len(dupl_val))]
allduplicates_test = [len([i for i, x in enumerate(notes_test) if x == dupl_test[j]]) for j in range(len(dupl_test))]

In [ ]:
import seaborn as sns

In [ ]:
sns.histplot(allduplicates_train)
sns.histplot(allduplicates_val)
sns.histplot(allduplicates_test)

In [ ]:
len(allduplicates_train), len(allduplicates_val), len(allduplicates_test)

In [ ]:
len(notes_train) - len(allduplicates_train), len(notes_val) - len(allduplicates_val), len(notes_test) - len(allduplicates_test)

In [ ]:
Counter(allduplicates_train), Counter(allduplicates_val), Counter(allduplicates_test)

In [ ]:
# get all indices, but the first

indices_train_remove = [i[1:] for i in indices_train]
indices_val_remove = [i[1:] for i in indices_val]
indices_test_remove = [i[1:] for i in indices_test]

indices_train_keep = [i[0] for i in indices_train]
indices_val_keep = [i[0] for i in indices_val]
indices_test_keep = [i[0] for i in indices_test]

In [ ]:
flat_list_train = np.asarray([item for sublist in indices_train_remove for item in sublist])
flat_list_val = np.asarray([item for sublist in indices_val_remove for item in sublist])
flat_list_test = np.asarray([item for sublist in indices_test_remove for item in sublist])

In [ ]:
assert len(flat_list_train) == len(set(flat_list_train))
assert len(flat_list_val) == len(set(flat_list_val))
assert len(flat_list_test) == len(set(flat_list_test))

In [ ]:
flat_list_train.sort()
flat_list_val.sort()
flat_list_test.sort()

indices_train_keep.sort()
indices_val_keep.sort()
indices_test_keep.sort()

In [ ]:
train_indices = np.arange(len(notes_train))
val_indices = np.arange(len(notes_val))
test_indices = np.arange(len(notes_test))

In [ ]:
# remove those indices from original data sets

In [ ]:
mask = np.ones(len(notes_train), bool)
mask[flat_list_train] = False
print(train_indices[mask], len(train_indices[mask]))
df_train_new = {}
df_train_new['notes'] = df_train['notes'][mask]
df_train_new['labels'] = df_train['labels'][mask]
df_train_new['inputs'] = df_train['inputs'][mask]

In [ ]:
mask = np.ones(len(notes_val), bool)
mask[flat_list_val] = False
print(val_indices[mask], len(val_indices[mask]))
df_val_new = {}
df_val_new['notes'] = df_val['notes'][mask]
df_val_new['labels'] = df_val['labels'][mask]
df_val_new['inputs'] = df_val['inputs'][mask]

In [ ]:
mask = np.ones(len(notes_test), bool)
mask[flat_list_test] = False
print(test_indices[mask], len(test_indices[mask]))
df_test_new = {}
df_test_new['notes'] = df_test['notes'][mask]
df_test_new['labels'] = df_test['labels'][mask]
df_test_new['inputs'] = df_test['inputs'][mask]